In [8]:
import os
import requests
import random
from bs4 import BeautifulSoup
import json
import pandas as pd
from jupyter_ui_poll import ui_events
from IPython.display import display, HTML, clear_output, Markdown
import time

# Set the seed for random operations
random.seed(1)

# Display the first part of the message
first_message = "<div style='font-size: 30px;'><strong>Welcome to the math test.</strong></div>"
display(HTML(first_message))
time.sleep(1.5)
clear_output(wait=True)

# Display the second part of the message
second_message = """
<div style='font-size: 20px;'>
This test will present arithmetic questions for 3 minutes. It gradually becomes more difficult as you progress. 
In case of a typo, such as entering something other than a number, one additional chance is provided.”
</div>
"""
display(HTML(second_message))
time.sleep(5)
clear_output(wait=True)

data_consent_info = """DATA CONSENT INFORMATION:
Please read:
we wish to record your response data
to an anonymised public data repository.
Your data will be used for educational teaching purposes practising data analysis and visualisation.
Please type yes in the box below if you consent to the upload."""
print(data_consent_info)
result = input("> ")
if result.lower() == "yes":  # This will handle 'yes', 'Yes', 'YES', etc.
    print("Thanks for your participation.")
    print("Please contact kwanhwi.lee.21@ucl.ac.uk")
    print("If you have any questions or concerns")
    print("regarding the stored results.")
else:
    # end code execution by raising an exception
    raise Exception("User did not consent to continue test.")
clear_output(wait=True)

id_instructions = """
Enter your anonymised ID
To generate an anonymous 4-letter unique user identifier please enter:
- two letters based on the initials (first and last name) of a childhood friend
- two letters based on the initials (first and last name) of a favourite actor / actress
e.g. if your friend was called Charlie Brown and film star was Tom
Cruise
then your unique identifer would be CBTC """
print(id_instructions)
user_id = input("Enter your anonymised ID: ")
user_age = input("Enter your age: ")
user_gender = input("Enter your gender (male or female): ")
user_coffee = input("Did you drink coffee in the last 6 hours (yes/no)? ")
user_test_time = input("At what time was the test taken? ")
user_condition = input("How is your mental clarity on a scale of 1 to 10? ")
are_you_night_owl = input("Are you a night owl (yes or no)? ")
clear_output(wait=True)

# Display start messages
start_messages = [
    "<div style='font-size: 30px;'><strong>Welcome to the math test.</strong></div>",
    "<div style='font-size: 20px;'>Get ready...</div>",
    "<div style='font-size: 26px;'><strong>Start!</strong></div>"
]

for message in start_messages:
    display(HTML(message))
    time.sleep(1.5)
    clear_output(wait=True)

# Define the questions and their correct answers
questions_with_answers = [
    ("9 + 6", 15),
    ("12 - 5", 7),
    ("9 × 5", 45),
    ("21 ÷ 7", 3),
    ("5²", 25),
    ("3³", 27),
    ("12 + 6 - 20", -2),
    ("16 - 7 + 27", 36),
    ("45 + 21 ÷ 3", 52),
    ("22 × 4 - 25", 63),
    ("5³ ÷ 5", 25),
    ("4² × 3 ÷ 16", 3),
    ("26 × 3 + 21 ÷ 3", 87),
    ("16 ÷ 2² × 5", 20),
    ("12² - 15 + 5", 134),
    ("240 - 145 ÷ 5", 229),
    ("64 - 4² + 14 ÷ 2", 50),
    ("(√25 ÷ 5)³", 1),
    ("√81² - 54 ÷ 8", 6567),
    ("452 - 244 ÷ 2 + 13", 239),
    ("(21 ÷ 9 - 25)⁰", 1),
    ("209 - 113 + 45 ÷ 5 - 27 ÷ 3", 88),
    ("1052 - 2149⁰ + 225 ÷ 15", 1077),
    ("886 - 5³ - 3 ÷ 3 + 20", 841),
    ("√46⁰ + 25 - 24", 2),
    ("241 × 44 × 0 + 435 - 245", 190),
    ("12 × 12 ÷ 2 + 5³ ÷ 5 - 114", -6),
    ("2166 - 352 + 24 ÷ 6 ÷ 2", 1820),
    ("1493 + 124 ÷ 2 + 44 + 67 × 0", 1569),
    ("24⁰ × 1⁰ ÷ 1", 1)
]

# Randomly generate additional questions from 15 to 30 if needed
additional_questions = questions_with_answers[14:30]

def ask_question(question_number, question_text, correct_answer):
    """Displays a question and allows two attempts for the user's answer."""
    for attempt in range(2):  # Two attempts
        clear_output(wait=True)  # Clear the previous question
        start_time = time.time()  # Define start time for the question attempt here
        # Display the question number and question on separate lines
        display(HTML(f"<div style='font-size: 24px;'>Question {question_number}:<br/>{question_text} = </div>"))
        try:
            user_answer = int(input())  # Get the answer from the user
            clear_output(wait=True)  # Clear the input prompt
            if user_answer == correct_answer:
                display(HTML(f"<div style='font-size: 24px; color: green;'>Correct!</div>"))
                time.sleep(0.5)  # Give time to see the 'Correct!' message
                return True, time.time() - start_time
            else:
                if attempt == 0:
                    display(HTML("<div style='font-size: 24px; color: red;'>Incorrect. You have one more trial to gain the score.</div>"))
                else:
                    display(HTML("<div style='font-size: 24px; color: red;'>Incorrect. Moving to next question.</div>"))
                time.sleep(1)  # Give time to see the 'Incorrect' message
        except ValueError:  # In case non-integer input is given
            if attempt == 0:
                display(HTML("<div style='font-size: 24px; color: red;'>Invalid input. You have one more trial to gain the score.</div>"))
            else:
                display(HTML("<div style='font-size: 24px; color: red;'>Invalid input. Moving to next question.</div>"))
            time.sleep(1.5)  # Give time to see the 'Invalid input' message
        if attempt == 1:  # After the second attempt, return None
            return False, time.time() - start_time


# Initialize the test
start_time = time.time()
total_time = 0
question_num = 1
correct_answers = 0

# Start the test and continue until 3 minutes have passed
while total_time < 180:
    # If all initial questions have been asked, pick random questions from 15 to 30
    if question_num > len(questions_with_answers):
        question, answer = random.choice(additional_questions)
    else:
        question, answer = questions_with_answers[question_num - 1]
    
    # Ask the question and get the time taken to answer
    correct, time_taken = ask_question(question_num, question, answer)
    total_time += time_taken
    if correct:
        correct_answers += 1
    question_num += 1

# Calculate the average time per question and update the data dictionary
average_time_per_question = total_time / (question_num - 1) if question_num > 1 else 0


# Upload to Google Form
# Define the data_dict here with all the collected information
data_dict = {
    "ID": user_id,  
    "age": user_age,
    "gender": user_gender,
    "total score": correct_answers,
    "average time taken per question": average_time_per_question,
    "Did you drink coffee in the last 6 hour(yes/no)": user_coffee,
    "At what time was the test taken?": user_test_time,
    "How is your mental clarity on a scale of 1 to 10": user_condition,
    "Are you a night owl?(yes/no)": are_you_night_owl
}

def send_to_google_form(data_dict, form_url):
    ''' Helper function to upload information to a corresponding google form 
        You are not expected to follow the code within this function!
    '''
    form_id = form_url[34:90]
    view_form_url = f'https://docs.google.com/forms/d/e/{form_id}/viewform'
    post_form_url = f'https://docs.google.com/forms/d/e/{form_id}/formResponse'

    page = requests.get(view_form_url)
    content = BeautifulSoup(page.content, "html.parser").find('script', type='text/javascript')
    content = content.text[27:-1]
    result = json.loads(content)[1][1]
    form_dict = {}
    
    loaded_all = True
    for item in result:
        if item[1] not in data_dict:
            print(f"Form item {item[1]} not found. Data not uploaded.")
            loaded_all = False
            return False
        form_dict[f'entry.{item[4][0][0]}'] = data_dict[item[1]]
    
    post_result = requests.post(post_form_url, data=form_dict)
    return post_result.ok
      
# Now create the DataFrame
dataframe = pd.DataFrame([data_dict])
myjson = dataframe.to_json()
newdf = pd.read_json(myjson)

# URL of the Google Form submission endpoint
forms_url = "https://docs.google.com/forms/d/e/1FAIpQLSfUuEbJeSjIdCP3JrrCskBiPZYMrxaZvFLno-IriDQXC5qVjQ/viewform"

# Send the data to the Google Form
send_to_google_form(data_dict, forms_url)

# Correctly assign values to score and questions_attempted
score = correct_answers
questions_attempted = question_num - 1

clear_output(wait=True)

# Use HTML display for the final message
final_message = f"""
<div style='font-size: 20px; text-align: center;'>
    <div>Test Completed.</div>
    <div>Score: {score}/{questions_attempted}</div>
    <div>Average Time per Question: {average_time_per_question:.2f} seconds/question</div>
    <div style='font-size: 24px; margin-top: 20px;'><strong>Thank you for your participation! I hope you enjoyed it</strong></div>
</div>
"""
display(HTML(final_message))
